In [1]:
'''
display_surfaceに関する処理
参考
https://qiita.com/mhangyo/items/76db7c6a6ebba6cf4330

dtypeに関する資料
http://discexuno.wp.xdomain.jp/machine-learning/python_fromfile_fromstring/
https://docs.python.org/3/library/struct.html

3Dfigure
https://morvanzhou.github.io/tutorials/data-manipulation/plt/3-5-3d/
'''
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2 
from scipy.fftpack import fft,ifft
import os
import glob
from pylab import *
import math
import matplotlib.animation as animation


In [2]:
'''
付録ーーデータの展開法

一つの実験についてのデータは四つのフォルダーに分けております。
そのうち、「Height」と書いてるフォルダーに変異データが入ってます。
そのフォルダーの中には縦方向（Y軸）の画素分のデータファイルがあります。
一つのファイルには一行についての画像枚分のデータが横方向（X軸）の画素分重ねております。

つまり、X行、Y棚、Fフレームのデータポイントを見つけるには
Y個目のファイルを開いて、X×フレーム数（16666frames=500fps×33.3s）＋F　のインデックスでXYFのデータがあります。

データ自体は64bit doubleです。バイトオーダーはビッグエンディアンです (byte order: big endian)。ヘッダーはないです。

どうぞよろしくお願いします。
ライオン

他の設定に関する情報元(resolution/framerate/downsampling,etc)　　\\FS1\Maedalab\home\junjie-hua\wave\fourD_sensor\rawdata\20191101\Captures
'''

#f = open(r"\\FS1\Maedalab\home\junjie-hua\wave\fourD_sensor\rawdata\20191101\test1_belly\Height\Height Row 0000.data",mode='rb')#0列目
#topo = np.fromfile(f, dtype='float64',sep='').reshape(256,20000) #float64の形で読み込むと，5120000の数列ができました．bellyの計測時間=20*1000msため，256*20000とreshapeしました．x軸上のサンプリング点が256個となります．




'\n付録ーーデータの展開法\n\n一つの実験についてのデータは四つのフォルダーに分けております。\nそのうち、「Height」と書いてるフォルダーに変異データが入ってます。\nそのフォルダーの中には縦方向（Y軸）の画素分のデータファイルがあります。\n一つのファイルには一行についての画像枚分のデータが横方向（X軸）の画素分重ねております。\n\nつまり、X行、Y棚、Fフレームのデータポイントを見つけるには\nY個目のファイルを開いて、X×フレーム数（16666frames=500fps×33.3s）＋F\u3000のインデックスでXYFのデータがあります。\n\nデータ自体は64bit doubleです。バイトオーダーはビッグエンディアンです (byte order: big endian)。ヘッダーはないです。\n\nどうぞよろしくお願いします。\nライオン\n\n他の設定に関する情報元(resolution/framerate/downsampling,etc)\u3000\u3000\\FS1\\Maedalab\\home\\junjie-hua\\wave\x0courD_sensor\rawdata\x8191101\\Captures\n'

In [3]:
#path = "//FS1//Maedalab//home//junjie-hua//wave//fourD_sensor//rawdata//20191101//test2_display_surface//Height" #display上の計測データ元　
path = "C://Users//huaju//Documents//data//fourD_sensor//20191101//test2_display//Height" #display上の計測データ元　

files = os.listdir(path) #all files in the faulder　縦方向(y軸)のデータファイル
s = []
xnum = 512
T = 16666
ynum = 0

#os.chdir("//FS1//Maedalab//home//junjie-hua//wave//fourD_sensor//rawdata//20191101//test2_display_surface//Height") #local()関数を利用するためのコードです
os.chdir("C://Users//huaju//Documents//data//fourD_sensor//20191101//test2_display//Height") #local()関数を利用するためのコードです

print("reading files")
for file in glob.glob("*.data"): 
    f = open(path + "//" + file ,mode='rb')#n列目　rb= read binary
    name = 'topo'+str(ynum)  #一つのファイルには一行についての画像枚分のデータが横方向（X軸）の画素分のデータを変数topo+str(n)に入れます
    locals()['topo'+str(ynum)]= np.fromfile(f, dtype='>d').reshape(xnum,T) 
    # https://docs.python.org/3/library/struct.html
    # ">" means big endian, "d" means double. 8532992の数列ができました．
    #print(file,"has ended")
    ynum = ynum + 1 
     
print(ynum ,"y-axis files") #y軸上サンプリング点の数を数えます--505個


reading files
505 y-axis files


In [4]:
'''
color mapのアニメーション
'''

# (x,y,t)のデータを取り出すための関数 
names = locals()
def f(x,y,t): 
    k = names['topo'+str(y)][x,t]  #topo_str(y)の数列から，(x,t)のデータを取り出します
    return k
#print(f(511,504,16666-1))
#print(f(505,0,0))


# X, Y value
x = np.linspace(0,xnum-1,xnum)
y = np.linspace(0,ynum-1,ynum)
X,Y = np.meshgrid(x,y)  #サンプリング点の数：x軸512個， y軸505個


#Z = [[0 for i in range(X.shape[1])] for i in range(Y.shape[0])]
Z = [[0 for row in range(xnum)] for col in range(ynum)]  #Z--x列y行    e.g. arr2D = [[ 0 for col in range(10) ] for row in range(4) ]-- 10列4行


print(" color map start")
for t in range(T):
    if t % 100 == 0:
        for y in range(ynum):#y軸
            for x in range(xnum):#x軸
                #print(x,y,t)
                Z[y][x] = f(x,y,t)     
        Z_array = np.array(Z)# turn list into array
        
       # color mapを作成する
        plt.pcolormesh(X, Y, Z_array, cmap='jet',vmin=-3,vmax=3) 
        pp=plt.colorbar (orientation="vertical") # カラーバーの表示 

        plt.xlabel('X', fontsize=18)
        plt.ylabel('Y', fontsize=18)
        plt.title("T = %s ms"% (t))
        plt.axes().set_aspect('equal')

        #print(t)
        #plt.savefig("//FS1//Maedalab//home//junjie-hua//wave//fourD_sensor//rawdata//20191101//Captures//animation//color_map//test2_display_surface//" + str(t) + ".png")
        plt.savefig("C://Users//huaju//Documents//data//fourD_sensor//20191101//Captures//animation//color_map//test2_display//global//" + str(t) + ".png")
        #plt.show()
        plt. close()
    else:
        continue

        
print("color map end")

 color map start


C:\Users\huaju\Miniconda3\lib\site-packages\ipykernel_launcher.py:40: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


color map end
